# Create a lookup table between date column index and other indices
(Excluding ID / Reponse columns)

In [1]:
import os
import re
import pickle
import time

import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

from scipy.sparse import csr_matrix

%matplotlib inline

# Custom modules
import const
import func

## Load data

In [2]:
const.TRAIN_FILES

['train_numeric', 'train_categorical_to_num', 'train_date']

In [3]:
[num_info, cat_info, date_info] = [func.get_station_info(f) for f in const.TRAIN_FILES]

# Date features refer to numeric/categorical features
date_info['ref_feat_nr'] = date_info['feature_nr'] - 1
date_info.columns = ['line','station','feat_nr_dat', 'name_dat','feature_nr']

In [4]:
date_info.head()

,line,station,feat_nr_dat,name_dat,feature_nr
0,0,0,1,L0_S0_D1,0
1,0,0,3,L0_S0_D3,2
2,0,0,5,L0_S0_D5,4
3,0,0,7,L0_S0_D7,6
4,0,0,9,L0_S0_D9,8


In [5]:
date_info.set_index(['line','station','feature_nr'], inplace=True)
num_info.set_index(['line','station','feature_nr'], inplace=True)
cat_info.set_index(['line','station','feature_nr'], inplace=True)

### Date info LUT

In [6]:
date_info.head()

feat_nr_dat  name_dat
line station feature_nr                       
0    0       0                     1  L0_S0_D1
             2                     3  L0_S0_D3
             4                     5  L0_S0_D5
             6                     7  L0_S0_D7
             8                     9  L0_S0_D9

In [7]:
num_info.columns = ['name_num']
cat_info.columns = ['name_cat']

In [8]:
num_info.head()

name_num
line station feature_nr          
0    0       0           L0_S0_F0
             2           L0_S0_F2
             4           L0_S0_F4
             6           L0_S0_F6
             8           L0_S0_F8

In [9]:
date_info = date_info.merge(cat_info,
                            how='outer',
                            left_index=True,
                            right_index=True). \
                      merge(num_info, 
                            how='outer',
                            left_index=True,
                            right_index=True). \
                      reset_index()
date_info.sample(10)

,line,station,feature_nr,feat_nr_dat,name_dat,name_cat,name_num
517,1,24,701,702.0,L1_S24_D702,L1_S24_F701,NaN
923,1,24,1239,NaN,NaN,L1_S24_F1239,NaN
2924,3,38,3955,NaN,NaN,L3_S38_F3955,NaN
179,0,10,245,246.0,L0_S10_D246,L0_S10_F245,NaN
1201,1,24,1602,NaN,NaN,L1_S24_F1602,NaN
1539,1,25,2049,NaN,NaN,L1_S25_F2049,NaN
1443,1,25,1928,NaN,NaN,L1_S25_F1928,NaN
241,0,11,326,NaN,NaN,NaN,L0_S11_F326
2511,3,29,3401,3402.0,L3_S29_D3402,NaN,L3_S29_F3401
70,0,3,101,102.0,L0_S3_D102,L0_S3_F101,NaN


In [10]:
'''
date_info = date_info.merge(cat_info[['feature_nr','name']], 
                            left_on='ref_feature_nr_dat', 
                            right_on='feature_nr', 
                            how='outer',
                            suffixes=('','_cat')). \
                      merge(num_info[['feature_nr','name']], 
                            left_on='ref_feature_nr_dat', 
                            right_on='feature_nr', 
                            how='outer',
                            suffixes=('','_num'))
'''

"\ndate_info = date_info.merge(cat_info[['feature_nr','name']], \n                            left_on='ref_feature_nr_dat', \n                            right_on='feature_nr', \n                            how='outer',\n                            suffixes=('','_cat')).                       merge(num_info[['feature_nr','name']], \n                            left_on='ref_feature_nr_dat', \n                            right_on='feature_nr', \n                            how='outer',\n                            suffixes=('','_num'))\n"

In [11]:
date_info.head(20)

,line,station,feature_nr,feat_nr_dat,name_dat,name_cat,name_num
0,0,0,0,1.0,L0_S0_D1,NaN,L0_S0_F0
1,0,0,2,3.0,L0_S0_D3,NaN,L0_S0_F2
2,0,0,4,5.0,L0_S0_D5,NaN,L0_S0_F4
3,0,0,6,7.0,L0_S0_D7,NaN,L0_S0_F6
4,0,0,8,9.0,L0_S0_D9,NaN,L0_S0_F8
5,0,0,10,11.0,L0_S0_D11,NaN,L0_S0_F10
6,0,0,12,13.0,L0_S0_D13,NaN,L0_S0_F12
7,0,0,14,15.0,L0_S0_D15,NaN,L0_S0_F14
8,0,0,16,17.0,L0_S0_D17,NaN,L0_S0_F16
9,0,0,18,19.0,L0_S0_D19,NaN,L0_S0_F18


### Update wrong numeric features

In [12]:
# Feature number L3_S37_D3942 is wrongly labeled, the feature belongs to L3 Station 36
# However, if you look at the timestamp values it belongs the something else...
# So the category label has the wrong label: L3_S36_F3941...

In [13]:
date_info.loc[(date_info.feature_nr==3941) & (date_info.station==37),['name_cat']] = ['L3_S36_F3941']
date_info.drop( date_info[(date_info.feature_nr==3941) & (date_info.station==36)].index, axis=0, inplace=True)

In [14]:
# If all are correctly labeled there shouldn't not be any duplicates
date_info.feature_nr.value_counts().head()

2047    1
1350    1
3395    1
1346    1
1344    1
Name: feature_nr, dtype: int64

### Save look-up table

In [15]:
date_info.to_csv(os.path.join(const.DATA_PATH,'date_feat_lut.csv'), index=False)